# Модули

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import torchvision.transforms.functional as TF

# Создание датасета

Класс Dataset основан на классе Dataset из модуля tourch

В нем мы задаем его объявление, расчёт длины и итерацию. Логика построения - объединение двух папок, каждая из которых представляет собой изображения своего класса.

In [3]:
class Dataset(torch.utils.data.Dataset):
  def __init__( self, papka_dir0:str, papka_dir1:str):
    super().__init__()

    self.papka_dir0 = papka_dir0
    self.papka_dir1 = papka_dir1

    self.dir0_list = sorted(os.listdir(papka_dir0))
    self.dir1_list = sorted(os.listdir(papka_dir1))

  def __len__(self):
    return len(self.dir0_list) + len(self.dir1_list)

  def __getitem__(self, index):

    class_id = 0 if index < len(self.dir0_list) else 1

    if class_id == 0:
      image_path = os.path.join(self.papka_dir0,
                                self.dir0_list[index])
    else:
      image_path = os.path.join(self.papka_dir1,
                                self.dir1_list[index- len(self.dir0_list)])

    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = TF.to_tensor(image)


    return image, class_id

In [6]:
train = Dataset('/content/drive/MyDrive/test/bit_1',
                '/content/drive/MyDrive/test/bit_0')
test = Dataset('/content/drive/MyDrive/train/bit_one',
               '/content/drive/MyDrive/train/bit_zero')

# Создание даталоудера

Создаем загрузчик датасетов, в котором задаем размеры батчей.

In [7]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(
    train, shuffle = True, batch_size = batch_size,
    drop_last=True, num_workers=1
    )
test_loader = torch.utils.data.DataLoader(
    test, shuffle = True, batch_size = batch_size,
    drop_last=False, num_workers=1
    )

# Архитектура сети

Класс Block - объединение двух слоев, основанное на логике сети ResNet

Класс ClassificatorNet - наша модель, состоящая из входного слоя, 5 блоков и выходного слоя.

In [8]:
class Block(nn.Module):
  def __init__(self, num_chanals):
    super().__init__()

    self.convalution0 = nn.Conv2d(num_chanals, num_chanals, 3, padding = 1)
    self.batch_norm0 = nn.BatchNorm2d(num_chanals)
    self.activation = nn.LeakyReLU(0.2, inplace= True)
    self.convalution1 = nn.Conv2d(num_chanals, num_chanals, 3, padding = 1)
    self.batch_norm1 = nn.BatchNorm2d(num_chanals)

  def forward(self, x):
    result = self.convalution0(x)
    result = self.batch_norm0(result)
    result = self.activation(result)
    result = self.convalution1(result)
    result = self.batch_norm1(result)

    return self.activation(x + result)

In [9]:
class ClassificatorNet(nn.Module):
  def __init__(self,in_ch, num_ch, out_ch):
    super().__init__()
    self.conv0 =  nn.Conv2d(in_ch, num_ch, 3, stride=2, padding= 1)
    self.activation0 = nn.LeakyReLU(0.2, inplace= True)


    self.layer1 = Block(num_ch)
    self.conv1 = nn.Conv2d(num_ch, num_ch, 1, stride=1, padding=1)
    self.layer2 = Block(num_ch)
    self.conv2 = nn.Conv2d(num_ch, 2*num_ch, 3, stride=2, padding= 1)
    self.layer3 = Block(num_ch*2)
    self.conv3 = nn.Conv2d(2*num_ch, 4*num_ch, 3, stride =2 , padding =1)
    self.layer4 = Block(num_ch*4)
    self.conv4 = nn.Conv2d(4*num_ch, 8*num_ch, 3, stride =2 , padding =1)
    self.layer5 = Block(num_ch*8)

    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.flatten = nn.Flatten()
    self.linear = nn.Linear(8*num_ch, out_ch)
    self.soft = nn.Softmax(1)

  def forward(self, x):
    result = self.conv0(x)
    result = self.activation0(result)

    result = self.layer1(result)
    result = self.conv1(result)
    result = self.layer2(result)
    result = self.conv2(result)
    result = self.layer3(result)
    result = self.conv3(result)
    result = self.layer4(result)
    result = self.conv4(result)
    result = self.layer5(result)

    result = self.avgpool(result)
    result = self.flatten(result)
    result = self.linear(result)

    return self.soft(result)


# Оптимайзер, лосс и метрики

Создание модели. Входных слоя 3 - RGB, количество слоев 1 блока согласно схеме 64, выходных 2.

Лосс - функция кросс энтропии.

Оптимайзер - Адам с linear rate 10^(-4).

Оптимайзер, его параметры и лосс функция выбраны согалсно статье.

In [10]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
model = ClassificatorNet(3, 64, 2)

count_parameters(model), model

In [12]:
loss_funk = nn.CrossEntropyLoss()
Optimizer = torch.optim.Adam(model.parameters(), lr= 0.0001)

In [13]:
def metric(pred, label):
  answer = F.softmax(pred.detach()).numpy().argmax(1) == label.numpy().argmax(1)
  return answer.mean()

# Тренировочный цикл

Обучение из 10 эпох, на датасете 3200 изображений. Для построения графиков записываем значения лосса и точность в массивы.

In [ ]:
epochs =  8
loss_epochs_list = []
acc_epochs_list = []
for epoch in range(epochs):
  loss_val = 0
  metr_val = 0

  for sample in (pbar := tqdm(train_loader)):
    img, label = sample[0], sample[1]
    Optimizer.zero_grad()

    label = F.one_hot(label, 2).float()
    pred = model(img)

    loss = loss_funk(pred, label)

    loss.backward()
    loss_item = loss.item()
    loss_val +=loss_item

    Optimizer.step()
    metr_current = metric(pred,label)
    metr_val += metr_current
  pbar.set_description(f'loss: {loss_item:.5f}, metric: {metr_current:.5f}')
  loss_epochs_list += [loss_val/len(train_loader)]
  acc_epochs_list += [metr_val/len(train_loader)]
  print(loss_epochs_list[-1])
  print(acc_epochs_list[-1])

In [ ]:
plt.title('Loss:')
plt.plot(loss_epochs_list)

In [ ]:
plt.title('Accuracy: ')
plt.plot(acc_epochs_list)

# Тесты

Тесты на выборке из 800 изображений. Модель на данном этапе не обучается.

In [ ]:
loss_val = 0
metr_val = 0

for sample in (pbar := tqdm(test_loader)):
  img, label = sample[0], sample[1]

  label = F.one_hot(label, 2).float()
  pred = model(img)
  loss = loss_funk(pred, label)
  loss_item = loss.item()
  loss_val +=loss_item

  metr_current = metric(pred,label)
  metr_val += metr_current
pbar.set_description(f'loss: {loss_item:.5f}, metric: {metr_current:.5f}')
print(loss_val/len(test_loader))
print(metr_val/len(test_loader))

Сохранение весов модели.

In [19]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/Model.pth')